# Sex check

We will impute the sex of each sample, and compare this with the self-reported gender.

Note the distinction between 'sex' and 'gender' used here: 'sex' refers to the genetic sex, where 'gender' is self-reported. We impute the sex by testing for the presence of a Y chromosome. The gender is obtained from a questionaire given to participants before a blood sample is drawn.

In [1]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '10g'}, tmp_dir='/home/olavur/tmp')

2021-10-15 09:10:19 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-10-15 09:10:19 WARN  Hail:37 - This Hail JAR was compiled for Spark 2.4.5, running with Spark 2.4.1.
  Compatibility is not guaranteed.
2021-10-15 09:10:20 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 2.4.1
SparkUI available at http://hms-beagle-6676655f87-9xllv:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/main/hail-20211015-0910-0.2.61-3c86d3ba497a.log


In [2]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models.scales import LogScale
output_notebook()

Loading BokehJS ...

In [3]:
import pandas as pd

In [4]:
BASE_DIR = '/home/olavur/experiments/2020-11-13_fargen1_exome_analysis'

## Load FarGen exome data

Load filtered, high-quality, variants.

In [17]:
mt = hl.read_matrix_table(BASE_DIR + '/data/mt/high_quality_variants.mt/')

In [18]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

Number of variants: 148305
Number of samples: 469


## Impute sex

We impute the sex of the samples by computing the inbreeding coefficient (F) on the X chromosome. This inbreeding coefficient is calculated as $F = \frac{O - E}{N-E}$ where $O$ is the observed number of homozygotes, $E$ is the expected number of homozygotes, and $N$ is the number of non-missing genotype calls. The expected number of homozygotes is calculated as $E = 1 - 2 f (1 - f)$ where $f$ is the minor-allel frequency.

In [20]:
imputed_sex_ht = hl.impute_sex(mt.GT)

Below we've plotted the inbreeding coefficient, and there is a quite clear clustering of individuals.

In [21]:
p = hl.plot.histogram(imputed_sex_ht.f_stat, title='Inbreeding coefficient (F) computed on the X chromosome')
p.plot_width = 800
p.plot_height = 500
show(p)

Based on the plot above, we define new $F$ thresholds for male and female, and do the imputation again.

In [22]:
imputed_sex_ht = hl.impute_sex(mt.GT, female_threshold=0.4, male_threshold=0.4)

# Make a new variable 'sex' that is either 'f' or 'm'.
imputed_sex_ht = imputed_sex_ht.annotate(sex = hl.if_else(imputed_sex_ht.is_female, 'f', 'm'))

## Load self-reported gender data

In [23]:
# Read the CSV.
gender_ht = hl.import_table(BASE_DIR + '/data/metadata/fargen_indi-gen.csv', delimiter=',')

# Key the table by individual name.
gender_ht = gender_ht.key_by(gender_ht.IndividualName)

# Us a boolean 'is_female' variable, like in the imputed data.
gender_ht = gender_ht.annotate(gender = hl.if_else(gender_ht.Gender == '0', 'f', 'm'))

# Recode the 'Gender' variable into a new 'gender' variable, that is either 'm', 'f', or missing.
gender_ht = gender_ht.transmute(gender = hl.case()
                                        .when(gender_ht.Gender == '0', 'f')
                                        .when(gender_ht.Gender == '1', 'm')
                                        .or_missing())

2021-10-15 09:10:55 Hail: INFO: Reading table without type imputation
  Loading field 'IndividualName' as type str (not specified)
  Loading field 'Gender' as type str (not specified)


## Compare imputed sex with self-reported gender

Below we compute a confusion matrix between self-reported gender and imputed sex.

We see that 7 samples have disconcordant sex and gender. Of these, 4 are reported as female and imputed as male, and 3 are reported as male and imputed as female.

In [24]:
# Annotate the imputed sex table with the self-reported gender.
imputed_sex_ht = imputed_sex_ht.annotate(gender=gender_ht[imputed_sex_ht.s].gender)

# Make Pandas series with the sex and gender.
sex = pd.Series(imputed_sex_ht.sex.collect(), name='Sex')
gender = pd.Series(imputed_sex_ht.gender.collect(), name='Gender')

# Calculate confusion matrix.
confusion_table = pd.crosstab(sex, gender, margins=True, margins_name='Sum')

confusion_table

2021-10-15 09:10:56 Hail: INFO: Coerced sorted dataset
2021-10-15 09:10:56 Hail: INFO: Coerced sorted dataset
2021-10-15 09:10:56 Hail: INFO: Coerced sorted dataset


Gender,f,m,Sum
Sex,,,
f,275,3,278
m,4,187,191
Sum,279,190,469


## Inspect disconcordant samples

We will inspect the samples where the self-reported gender is different from the imputed sex.

As we have seen, the males and females cluster very nicely w.r.t. the inbreeding coeffcient, so we can be confident that we are correctly imputing the sex. We used imputed samples with $F > 0.4$ as male and $F < 0.4$ as female.

Below we see the imputation data for the samples where the imputed sex and self-reported gender mismatch. Note that these samples fall nicely into the categories defined by the cut-off values we defined.

In [25]:
imputed_sex_ht.filter(imputed_sex_ht.sex != imputed_sex_ht.gender).show()

2021-10-15 09:10:57 Hail: INFO: Coerced sorted dataset
2021-10-15 09:10:57 Hail: INFO: Coerced sorted dataset


,,,,,,,
s,is_female,f_stat,n_called,expected_homs,observed_homs,sex,gender
str,bool,float64,int64,float64,int64,str,str
"""FN000187""",true,-8.52e-02,1185,1.01e+03,992,"""f""","""m"""
"""FN000861""",true,-1.80e-01,1284,1.08e+03,1046,"""f""","""m"""
"""FN000871""",true,-8.36e-02,1282,1.08e+03,1062,"""f""","""m"""
"""FN000884""",false,8.70e-01,982,8.59e+02,966,"""m""","""f"""
"""FN000902""",false,8.82e-01,716,6.48e+02,708,"""m""","""f"""
"""FN000957""",false,8.75e-01,920,8.08e+02,906,"""m""","""f"""
"""FN001127""",false,9.22e-01,997,8.68e+02,987,"""m""","""f"""


We will investigate whether poor data quality can explain the disconcordancy. If anything seems to be abnormal with these samples, we may distrust the data.

In [26]:
# Annotate the table with a boolean sex mismatch variable.
imputed_sex_ht = imputed_sex_ht.annotate(sex_mismatch = imputed_sex_ht.sex != imputed_sex_ht.gender)

# Annotate the matrix table with the same information.
mt = mt.annotate_cols(sex_mismatch = imputed_sex_ht[mt.s].sex_mismatch)

The plot below shows the per samples DP mean and against different other QC variables. The samples with disconcordant sex are highlighted in blue.

We must note that it seems that GQ seems to be correlated with number of heterozygotes. Samples with low genotype quality seem to have fewer heterozygotes. This does make some intuitivt sense. If a sample has low depth it will have fewer called variants, therefore more homozygote reference and therefore fewer heterozygotes. This same pattern is reflected in the call rate and number of singletons.

In [27]:
exprs_list = [('# heterozygotes', mt.sample_qc.n_het), ('Ti/Tv rate', mt.sample_qc.r_ti_tv), ('Call rate', mt.sample_qc.call_rate), ('# singletons', mt.sample_qc.n_singleton)]
plot_list = []
for name, exprs in exprs_list:
    p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, exprs, label=mt.sex_mismatch, title=name, xlabel='DP mean', ylabel=name)
    p.plot_width = 800
    p.plot_height = 500
    plot_list.append(p)

2021-10-15 09:10:58 Hail: INFO: Coerced sorted dataset
2021-10-15 09:10:58 Hail: INFO: Coerced sorted dataset
2021-10-15 09:10:59 Hail: INFO: Coerced sorted dataset
2021-10-15 09:10:59 Hail: INFO: Coerced sorted dataset
2021-10-15 09:11:00 Hail: INFO: Coerced sorted dataset
2021-10-15 09:11:00 Hail: INFO: Coerced sorted dataset
2021-10-15 09:11:01 Hail: INFO: Coerced sorted dataset
2021-10-15 09:11:01 Hail: INFO: Coerced sorted dataset


In [28]:
show(gridplot(plot_list, ncols=2, plot_width=600, plot_height=400))

There seems to be nothing abnormal with these samples with disconcordant imputed sex and self-reported gender. We will therefore trust that our computation is correct, and keep the samples.